In [ ]:
#SVMLinear
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
import joblib

# dataset
filename = '/content/SingleTweets.csv'
dataset = pd.read_csv(filename)

X = dataset.iloc[:, 1:22].values
y = dataset.iloc[:, 22].values

# Hyperparameters
svm_parameters = {'C': [10**i for i in range(-3, 0, 10)]}

# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Strategy
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Initialize
svm_linear_accuracy_scores = []
svm_linear_precision_scores = []
svm_linear_recall_scores = []
svm_linear_f1_scores = []

# best model
best_svm_linear_model = None
best_svm_linear_score = 0.0

# Cross-validation
for train_index, test_index in outer_cv.split(X_train, y_train):
    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

    # SVM Linear
    svm_linear_classifier = SVC(kernel='linear', random_state=42)
    svm_linear_grid_search = GridSearchCV(svm_linear_classifier, svm_parameters, cv=inner_cv)
    svm_linear_grid_search.fit(X_train_fold, y_train_fold)
    svm_linear_best_params = svm_linear_grid_search.best_params_
    svm_linear_best_model = SVC(kernel='linear', random_state=42, **svm_linear_best_params)

    # Fit best model on full training set
    svm_linear_best_model.fit(X_train_fold, y_train_fold)

    # test set
    svm_linear_predictions = svm_linear_best_model.predict(X_test_fold)

    # performance metrics
    svm_linear_accuracy_scores.append(accuracy_score(y_test_fold, svm_linear_predictions))
    svm_linear_precision_scores.append(precision_score(y_test_fold, svm_linear_predictions))
    svm_linear_recall_scores.append(recall_score(y_test_fold, svm_linear_predictions))
    svm_linear_f1_scores.append(f1_score(y_test_fold, svm_linear_predictions))

    # Store the best model
    current_score = accuracy_score(y_test_fold, svm_linear_predictions)
    if current_score > best_svm_linear_score:
        best_svm_linear_score = current_score
        best_svm_linear_model = svm_linear_best_model

# Save best model
if best_svm_linear_model is not None:
    joblib.dump(best_svm_linear_model, 'best_svm_linear_model.pkl')

# Calculate on test set
svm_linear_average_accuracy = np.mean(svm_linear_accuracy_scores)
svm_linear_average_precision = np.mean(svm_linear_precision_scores)
svm_linear_average_recall = np.mean(svm_linear_recall_scores)
svm_linear_average_f1 = np.mean(svm_linear_f1_scores)

# Print
print("\nSVM Linear Test Set:")
print("Average Accuracy:", svm_linear_average_accuracy)
print("Average Precision:", svm_linear_average_precision)
print("Average Recall:", svm_linear_average_recall)
print("Average F1-score:", svm_linear_average_f1)



SVM Linear Test Set:
Average Accuracy: 0.85375
Average Precision: 0.8627143783010578
Average Recall: 0.8413074568725655
Average F1-score: 0.8517582422512969
